In [1]:
import json
import os
import gzip
from collections import Iterable
import configparser
from pysenal import read_jsonline, write_json, append_line

_ENCODING_UTF8 = 'utf-8'

<ipython-input-1-0b9e6023b198>:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable


In [32]:
def houchuli(src, tgt):
    dict_list = read_jsonline(src)
    for dd in dict_list:
        shou = []
        remove_list = []
        for key in dd['given_keyword_tokens']:
            if len(key) != 0:
                shou.append(key[0])
        for key in dd['pred_keyphrases']:
            if key[0] in shou:
                remove_list.append(key)
        for remove in remove_list:
            dd['pred_keyphrases'].remove(remove)
    for dd in dict_list:
        filename = tgt
        append_line(filename, json.dumps(dd))
        
        
datasets = ['inspec', 'krapivin', 'nus', 'semeval', 'kp20k']
for data in datasets:
    src = '/mnt/KeyphraseExpansion/data/jsonl/'+data+'/'+data+'_pred.jsonl'
    tgt = '/mnt/KeyphraseExpansion/data/jsonl/'+data+'/'+data+'_pred_hou.jsonl'
    houchuli(src, tgt)

In [33]:
from evaluation import KeyphraseEvaluator
eval_topn = (5, 10)
for data in datasets:
    pred_filename = '/mnt/KeyphraseExpansion/data/jsonl/'+data+'/'+data+'_pred_hou.jsonl'
    macro_evaluator = KeyphraseEvaluator(eval_topn, 'macro', 'title_and_abstract_tokens', 'keyword_tokens')
    macro_all_ret = macro_evaluator.evaluate(pred_filename)
    print(data)
    print('all',macro_all_ret[5]['f1'])
    print('all',macro_all_ret[10]['f1'])

inspec
all 0.21207773287294557
all 0.21806019872200694
krapivin
all 0.16063306331855168
all 0.1379028933051004
nus
all 0.17777682249664215
all 0.17340667452404843
semeval
all 0.1204180065394617
all 0.1406252943764692
kp20k
all 0.1807794833769905
all 0.15376315713925598


In [28]:
macro_all_ret[5]['f1']

0.18077948337703306